This notebook compares `DopplerCIB` with `galCIB` and checks for any issues. `DopplerCIB` is well-tested and `galCIB` is a derivative of the codebase. 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/Users/tkarim/opt/anaconda3/envs/cosmo/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
# galCIB modules

import consts
import powerspectra as ps
import halo
import gal 
import precalc as pc
import cib
import cosmology

(210,)
Note: redshifts have been re-sorted (earliest first)


/Users/tkarim/research/galCIB/powerspectra.py:63: RuntimeWarning: divide by zero encountered in log10
  ELL_sampled = np.logspace(np.log10(consts.LMIN),
/Users/tkarim/opt/anaconda3/envs/cosmo/lib/python3.10/site-packages/numpy/core/function_base.py:157: RuntimeWarning: invalid value encountered in multiply
  y *= step
/Users/tkarim/opt/anaconda3/envs/cosmo/lib/python3.10/site-packages/numpy/core/function_base.py:167: RuntimeWarning: invalid value encountered in add
  y += start


In [4]:
# DopplerCIB modules

import sys 
sys.path.append("//Users/tkarim/research/DopplerCIB/")

import CIB_halo as mcib
import cosmo_related as mcosmo
import input_var_cibmean as minput
import Gal_halo as mgh
import CIBxGal_halo as mcg
import hmf_unfw_bias as mhmf_unfw

First initialize relevant `DopplerCIB` objects

In [5]:
## initialize cosmo_var_iv and data_var_iv to pass to DopplerCIB

cosmo_var = mcosmo.cosmo_var_iv(mass=consts.Mh_Msol,
                                z=consts.Plin['z'],
                                do_powerspec=1)

# experimental specification 

cib_exp = 'Planck'
# color corrections for 100, 143, 217, 353, 545, 857 and 3000 GHz for Planck
cc_pl = np.array([1.076, 1.017, 1.119, 1.097, 1.068, 0.995])  # , 0.960])
cc = cc_pl  

nu0 = np.array([100., 143., 217., 353., 545., 857.]) 
nus = ['100', '143', '217', '353', '545', '857']
# fwhm obtained from https://wiki.cosmos.esa.int/planckpla/index.php/Effective_Beams
fwhm = np.array([9.651, 7.248, 4.990, 4.818, 4.682, 4.325])  # in arcmin
fwhm_rad = fwhm*(np.pi/180.)/60.
# fwhm = 4.8  # 4.8 Planck arcmin # 0.5 CCAT-Prime
# 1303.5067 for sensitivity (Tab:4 muK deg & kJy/sr deg)
sensitivity_muKarcmin = np.array([108., 48., 60., 210., 1137., 29075.])
sensitivity_Jy2_sr = np.array([58., 26.929, 72., 305, 369., 369.])
sensitivity_Jy_sr = np.sqrt(sensitivity_Jy2_sr)
# sensitivity = 13.5  # 13.5 Planck Jy/sr  # 1.2 CCAT-Prime
ell = np.logspace(2, np.log10(2000), 99)
fc = np.ones(len(nu0))

strfig = "allcomponents_lognormal_sigevol_1p5zcutoff_nolens_onlyautoshotpar_no3000_gaussian600n857n1200_planck_spire_hmflog10.txt"
cibres = "/Users/tkarim/research/galCIB/data/one_halo_bestfit_"+strfig

exp = {'name': cib_exp,
          'do_cibmean': 0,
          'cc': cc,
          'fc': fc,
          'snuaddr': '/Users/tkarim/research/galCIB/data/filtered_snu_planck.fits',
          # 'nu0min': nu0min, 'nu0max': nu0max,
          # 'nucen': str(int(nucen)),
          'nu0': nu0,
          'ell': ell,
          'fwhm': fwhm,
          'sensitivity': sensitivity_Jy_sr,
          'cibpar_resfile': cibres,
          'nu_string': nus}

## initialize data_var_iv 
data_var_iv = minput.data_var_iv(exp = exp)

## initialize CIBXGal_halo.py
mcibgal_obj_pk = mcg.CIBxgal(data_var_iv=data_var_iv,
                          cosmo_var_iv=cosmo_var,
                          gal_exp='DESI_ELG',
                          r_l=1)
mgal_obj = mgh.ProfHODMore15(data_var_iv, cosmo_var, gal_exp='DESI_ELG')

Calculating the halo mass function for given mass and redshift for CIB mean calculations.
[39.52037275]
5.298428495800479e-17
2.4832862197860225e-18
